In [15]:
import requests
from bs4 import BeautifulSoup

def crawl_vnexpress_articles(num_articles=50):
    url = "https://vnexpress.net"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    articles = []
    links = []

    # Lấy các thẻ chứa đường link bài viết chính
    for a in soup.select("a[href^='https://vnexpress.net']"):
        href = a.get("href")
        if href and href not in links and len(links) < num_articles:
            links.append(href)

    print(f"Đã thu thập {len(links)} link bài viết.")

    for link in links:
        try:
            r = requests.get(link, timeout=5)
            s = BeautifulSoup(r.text, "html.parser")

            # Lấy nội dung chính (thẻ article)
            body = s.find("article")
            if body:
                paragraphs = body.find_all("p")
                content = "\n".join([p.get_text() for p in paragraphs])
                if len(content) > 100:
                    articles.append(content)
        except:
            continue

    # Gộp toàn bộ nội dung và lưu
    full_text = "\n".join(articles)
    with open("raw_articles.txt", "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"Đã lưu {len(articles)} bài viết vào file raw_articles.txt")

# Gọi hàm crawl
crawl_vnexpress_articles()


Đã thu thập 50 link bài viết.
Đã lưu 48 bài viết vào file raw_articles.txt


In [16]:
import re

def clean_and_tokenize(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 1. Chuyển về chữ thường
    text = text.lower()

    # 2. Loại bỏ số và ký tự đặc biệt (giữ lại chữ cái và khoảng trắng)
    text = re.sub(r'[^a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ'
                  r'ìíịỉĩòóọỏõôồốộổỗơờớợởỡ'
                  r'ùúụủũưừứựửữỳýỵỷỹđ\s]', '', text)

    # 3. Loại bỏ nhiều khoảng trắng dư thừa
    text = re.sub(r'\s+', ' ', text).strip()

    # 4. Tách từ
    tokens = text.split()

    print(f"Tổng số từ sau khi tách: {len(tokens)}")
    return tokens

# Gọi thử
tokens = clean_and_tokenize('raw_articles.txt')

# (Tùy chọn) Lưu kết quả ra file để xem
with open('tokens.txt', 'w', encoding='utf-8') as f:
    for word in tokens:
        f.write(word + '\n')


Tổng số từ sau khi tách: 38979


In [17]:
from underthesea import word_tokenize

def clean_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    
    # Giữ lại chữ cái tiếng Việt và khoảng trắng
    text = re.sub(r'[^a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ'
                  r'ìíịỉĩòóọỏõôồốộổỗơờớợởỡ'
                  r'ùúụủũưừứựửữỳýỵỷỹđ\s]', '', text)
    
    # Loại bỏ khoảng trắng dư
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_with_underthesea(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        raw_text = f.read()

    cleaned = clean_text(raw_text)

    # Tách từ bằng underthesea (giữ đúng từ ghép)
    tokenized_text = word_tokenize(cleaned, format="text")  # ra dạng: "trí_tuệ nhân_tạo là ..."
    
    # Chuyển về danh sách từ (loại bỏ dấu _ nếu muốn giữ lại dạng "trí tuệ")
    tokens = tokenized_text.replace("_", " ").split()
    
    print(f"Đã tách được {len(tokens)} từ.")
    return tokens

# Gọi hàm
tokens = tokenize_with_underthesea("raw_articles.txt")

# Lưu ra file nếu cần
with open("tokens_underthesea.txt", "w", encoding="utf-8") as f:
    for word in tokens:
        f.write(word + '\n')


Đã tách được 38979 từ.


In [18]:
import os

def load_all_vietnamese_words(folder_path):
    all_words = set()
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    word = line.strip().lower()
                    if word:
                        all_words.add(word)
    return all_words


In [19]:
def filter_vietnamese_words(tokens, vietnamese_dict):
    return [word for word in tokens if word in vietnamese_dict]


In [20]:
# Đường dẫn đến thư mục từ điển
folder_path = 'tu_dien_goc'

# 1. Tải từ điển tiếng Việt từ tất cả file
vietnamese_dict = load_all_vietnamese_words(folder_path)

# 2. Giả sử bạn có tokens từ file tokens_underthesea.txt
with open('tokens_underthesea.txt', 'r', encoding='utf-8') as f:
    tokens = [line.strip().lower() for line in f if line.strip()]

# 3. Lọc từ
vietnamese_only_tokens = filter_vietnamese_words(tokens, vietnamese_dict)

# 4. Loại bỏ trùng lặp bằng set()
unique_tokens = sorted(set(vietnamese_only_tokens))

# 5. Ghi ra file
with open('vietnamese_only_tokens.txt', 'w', encoding='utf-8') as f:
    for word in vietnamese_only_tokens:
        f.write(word + '\n')

print(f"Đã lọc {len(vietnamese_only_tokens)} từ tiếng Việt từ tổng {len(tokens)} từ.")


Đã lọc 37194 từ tiếng Việt từ tổng 38979 từ.
